In [1]:
"""
Earth Engine authentication
"""

import ee 

ee.Authenticate() 
ee.Initialize()

Enter verification code: 4/1AY0e-g6UOg-7I8QF1N1Ffpm8qcMb15C7Za7bnFZvq5b4JRZWevAZ6v-0qQQ

Successfully saved authorization token.


In [2]:
import matplotlib.pyplot as plt 
import numpy as np 
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp 
import time
%matplotlib inline

In [3]:
"""
Folium map configuration
"""
import folium 

def add_ee_layer(self, ee_image_object, vis_params, name):
    """
    display EE image tiles
    """
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
      ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [42]:
# region coordinates from geojson.io
# geoJSON = {
#   "type": "FeatureCollection",
#   "features": [
#     {
#       "type": "Feature",
#       "properties": {},
#       "geometry": {
#         "type": "Polygon",
#         "coordinates": [
#           [
#             [
#               -71.62519454956053,
#               44.168106976083905
#             ],
#             [
#               -71.59412384033203,
#               44.168106976083905
#             ],
#             [
#               -71.59412384033203,
#               44.1956821981841
#             ],
#             [
#               -71.62519454956053,
#               44.1956821981841
#             ],
#             [
#               -71.62519454956053,
#               44.168106976083905
#             ]
#           ]
#         ]
#       }
#     }
#   ]
# }

geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -71.66107177734375,
              44.148464079182745
            ],
            [
              -71.51687622070312,
              44.148464079182745
            ],
            [
              -71.51687622070312,
              44.21420205653367
            ],
            [
              -71.66107177734375,
              44.21420205653367
            ],
            [
              -71.66107177734375,
              44.148464079182745
            ]
          ]
        ]
      }
    }
  ]
}


coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


In [43]:
"""
Get Sentinel-1 imagery for our region of interest
"""

#decibel
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))

# float
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))

# display bands
ffa_db.bandNames().getInfo()

['VV', 'VH', 'angle']

In [44]:
"""
Display a VV image
"""
url = ffa_db.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [45]:
"""
RGB composite using polarization
"""

# Folium wants long-lat instead of lat-long -- reverse the list!
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(ffa_db.select('VV'),
                   ffa_db.select('VH'),
                   ffa_db.select('VV').divide(ffa_db.select('VH')))

# Create the map object.
m = folium.Map(location=location, zoom_start=14, height=800, width=1000)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)


In [46]:
"""
Images for change detection 
-important that images have same incidence angle --> specify asc/desc
"""

im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                .filterBounds(aoi)
                .filterDate(ee.Date('2020-08-01'),ee.Date('2020-08-31'))
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                .filter(ee.Filter.eq('relativeOrbitNumber_start', 62))
                .sort('system:time_start'))

# acquisition times
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]


['08/02/20', '08/14/20', '08/26/20']

In [47]:
im_list = im_coll.toList(im_coll.size())
im1 = ee.Image(im_list.get(0)).select('VV').clip(aoi)
im2 = ee.Image(im_list.get(1)).select('VV').clip(aoi)
ratio = im1.divide(im2)

location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=12, height=800, width=1000)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_child(folium.LayerControl())

display(mp)


In [48]:
m=5

# Decision threshold alpha/2:
dt = f.ppf(0.0005, 2*m, 2*m)

# LRT statistics.
q1 = im1.divide(im2)
q2 = im2.divide(im1)

# Change map with 0 = no change, 1 = decrease, 2 = increase in intensity.
c_map = im1.multiply(0).where(q2.lt(dt), 1)
c_map = c_map.where(q1.lt(dt), 2)

# Mask no-change pixels.
c_map = c_map.updateMask(c_map.gt(0))

# Display map with red for increase and blue for decrease in intensity.
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(
    location=location, 
    zoom_start=13, height=800, width=1000)
folium.TileLayer('OpenStreetMap').add_to(mp)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_ee_layer(c_map,
                {'min': 0, 'max': 2, 'palette': ['black', 'blue', 'red']},
                'Change Map')
mp.add_child(folium.LayerControl())

display(mp)


## Bivariate Change

In [49]:
# region coordinates from geojson.io
# geoJSON = {
#   "type": "FeatureCollection",
#   "features": [
#     {
#       "type": "Feature",
#       "properties": {},
#       "geometry": {
#         "type": "Polygon",
#         "coordinates": [
#           [
#             [
#               -71.62519454956053,
#               44.168106976083905
#             ],
#             [
#               -71.59412384033203,
#               44.168106976083905
#             ],
#             [
#               -71.59412384033203,
#               44.1956821981841
#             ],
#             [
#               -71.62519454956053,
#               44.1956821981841
#             ],
#             [
#               -71.62519454956053,
#               44.168106976083905
#             ]
#           ]
#         ]
#       }
#     }
#   ]
# }

geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -71.66107177734375,
              44.148464079182745
            ],
            [
              -71.51687622070312,
              44.148464079182745
            ],
            [
              -71.51687622070312,
              44.21420205653367
            ],
            [
              -71.66107177734375,
              44.21420205653367
            ],
            [
              -71.66107177734375,
              44.148464079182745
            ]
          ]
        ]
      }
    }
  ]
}

coords = geoJSON['features'][0]['geometry']['coordinates']
aoi1 = ee.Geometry.Polygon(coords)


In [50]:
im1 = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                .filterBounds(aoi1)
                .filterDate(ee.Date('2016-08-01'), ee.Date('2016-08-31'))
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                .filter(ee.Filter.eq('relativeOrbitNumber_start', 62))
                .first()
                .clip(aoi1))
im2 = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterBounds(aoi1)
                .filterDate(ee.Date('2019-08-01'), ee.Date('2019-08-31'))
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                .filter(ee.Filter.eq('relativeOrbitNumber_start', 62))
                .first()
                .clip(aoi1))

acq_time = im1.get('system:time_start').getInfo()
print( time.strftime('%x', time.gmtime(acq_time/1000)) )
acq_time = im2.get('system:time_start').getInfo()
print( time.strftime('%x', time.gmtime(acq_time/1000)) )


08/11/16
08/08/19


In [51]:

det = lambda im: im.expression('b(0)*b(1)')
chi2cdf = lambda chi2, df: ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

# The observed test statistic image -2logq.
m2logq = det(im1).log().add(det(im2).log()).subtract(
    det(im1.add(im2)).log().multiply(2)).add(4*np.log(2)).multiply(-2*m)

# The P value image prob(m2logQ > m2logq) = 1 - prob(m2logQ < m2logq).
p_value = ee.Image.constant(1).subtract(chi2cdf(m2logq, 2))

# Project onto map.
location = aoi1.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=14, height=800, width=1000)
mp.add_ee_layer(p_value,
                {'min': 0,'max': 1, 'palette': ['black', 'white']}, 'P-value')
mp.add_child(folium.LayerControl())


In [71]:
c_map = p_value.multiply(0).where(p_value.lt(0.01), 1)


mp = folium.Map(location=location, zoom_start=12, height=800, width=1000)
# mp.add_ee_layer(background.select('HH_mean'), {}, 'background')
mp.add_ee_layer(ffa_db.select('VV'), {'min':-20, 'max':0}, 'background')
# mp.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')
mp.add_ee_layer(c_map.updateMask(
    c_map.gt(0)), {'min': 0, 'max': 1, 'palette': ['black', 'red']}, 'c_map')
mp.add_child(folium.LayerControl())
